In [1]:
import os
import pysam
from typing import Optional, Dict
import time

In [2]:
cram_path="/nas/longleaf/home/catererz/epi/lpa/cram/NWD340659.b38.irc.v1.cram"
crai_path="/nas/longleaf/home/catererz/epi/lpa/cram/NWD340659.b38.irc.v1.cram.crai"

In [3]:
def subset_cram(
    input_cram: str,
    output_bam: str,
    region: Dict[str, int],
    input_crai: Optional[str] = None,
    reference_fasta: Optional[str] = None
):
    """
    Subset a CRAM file to a specific region and save as BAM.

    :param input_cram: Path to the input CRAM file.
    :param output_bam: Path to the output BAM file.
    :param region: Dictionary with 'chrom', 'start', and 'end' keys.
    :param input_crai: Path to the CRAM index file (optional).
    :param reference_fasta: Path to the reference FASTA file (optional).
    :return: Path to the created BAM file.
    """

    # Build kwargs for AlignmentFile depending on what's provided
    open_kwargs = {"reference_filename": reference_fasta} if reference_fasta else {}
    if input_crai:
        open_kwargs["index_filename"] = input_crai

    # Open input CRAM
    infile = pysam.AlignmentFile(input_cram, "rc", **open_kwargs)

    # Create output BAM file
    outfile = pysam.AlignmentFile(output_bam, "wb", header=infile.header)

    # Fetch and write reads in specified region
    for read in infile.fetch(region["chrom"], region["start"], region["end"]):
        outfile.write(read)

    infile.close()
    outfile.close()

    # Index the new BAM file
    pysam.index(output_bam)

    return output_bam

# subset_cram(
#     input_cram="input.cram",
#     output_bam="subset.bam",
#     region={"chrom": "chr6", "start": 159900000, "end": 161100000},
#     reference_fasta="hg38.fa"
# )


In [5]:
# Time the runs
start = time.time()
subset_cram(
    input_cram=cram_path, 
    output_bam="/nas/longleaf/home/catererz/epi/lpa/cram/no_ref.bam", 
    region={"chrom": "chr6", "start": 159900000, "end": 161100000}
)

no_ref_time = time.time() - start

start = time.time()
subset_cram(
    input_cram=cram_path, 
    output_bam="/nas/longleaf/home/catererz/epi/lpa/cram/with_ref.bam", 
    region={"chrom": "chr6", "start": 159900000, "end": 161100000},
    reference_fasta="/nas/longleaf/home/catererz/epi/lpa/cram/hg38.fa"
)
with_ref_time = time.time() - start

# Compare BAM sizes
no_ref_size = os.path.getsize("/nas/longleaf/home/catererz/epi/lpa/cram/no_ref.bam")
with_ref_size = os.path.getsize("/nas/longleaf/home/catererz/epi/lpa/cram/with_ref.bam")

print(f"  No reference: {no_ref_time:.2f}s, {no_ref_size} bytes")
print(f"With reference: {with_ref_time:.2f}s, {with_ref_size} bytes")


  No reference: 4.38s, 19858072 bytes
With reference: 3.79s, 19858072 bytes


## **1. Cohort Filtering & QC**

* **Primary dataset**: UK Biobank exome sequencing + SNP array.
* **Ancestry & relatedness filters**

  * Start with White British for discovery.
  * Remove PC outliers (>6 SD from mean of any of first 10 PCs).
  * Remove one from each ≤2nd-degree related pair (kinship >0.0884), keeping individuals with relevant phenotype measurements.
  * Use expanded European-ancestry sample for follow-up.
* **Phenotype of interest**: Serum lipoprotein(a) (Lp(a)), with special handling for values outside reportable range (crop to 3.7 or 190 nmol/L, keep binary info).

---

## **2. VNTR Identification & Genotyping (KIV-2 Specific)**

1. **Identify KIV-2 repeat**

   * Custom algorithm to find large multi-kb repeats in GRCh38 (could not rely on read-spanning approaches because UKB WES has 76 bp reads).
2. **Estimate diploid VNTR length** from exome depth-of-coverage:

   * Count reads overlapping KIV-2.
   * Normalize per-sample coverage using 200 most similar exomes to reduce technical bias.
3. **Phase & impute VNTR allele lengths**:

   * Statistical phasing with surrounding SNP haplotypes.
   * Impute phased VNTR allele lengths into full UKB cohort.
4. **Genotype PSVs (Paralogous Sequence Variants) within KIV-2**:

   * Catalog within-repeat sequence variation from exome data.
   * Estimate PSV copy numbers and incorporate into phasing/imputation.

---

## **3. Fine-Mapping & Likely-Causal Variant Identification**

* Combine:

  * **Phased KIV-2 allele lengths**
  * **PSVs in and near KIV-2 exons**
  * **SNPs/indels in *LPA***
* Special “effective haploid” fine-mapping:

  * Use carriers of null/very-low Lp(a) alleles to isolate effects of variants on single chromosomes.
* Identify **\~23 likely-causal LPA variants** (protein-altering or 5’UTR) via stepwise conditional analysis.

---

## **4. Predictive Modeling of Lp(a)**

* Build an allele-level model:

  * Baseline curve: nonlinear relationship between KIV-2 length and Lp(a) in absence of modifiers.
  * Add multiplicative effects of causal SNPs/PSVs on that baseline per haplotype.
* Compare models:

  * **KIV-2 length only**
  * **Length + causal variants (linear model)**
  * **Full nonlinear haplotype model** (final).

---

## **5. Association Testing**

* For KIV-2 VNTR & Lp(a):

  * Association analysis in BOLT-LMM with standard covariates.
  * Fine-mapping with FINEMAP and SuSiE to confirm causality (PIP=1 for KIV-2 in Lp(a) model).
* Downstream: logistic regression for myocardial infarction and T2D risk using genetically predicted Lp(a).

---

## **6. PRS Construction (Your Goal)**

To make a VNTR-informed PRS from their framework:

1. **Genotype & phase KIV-2 VNTR lengths** in your target cohort (follow their depth normalization, phasing, imputation strategy).
2. **Genotype key LPA causal variants** (the 23 they identify, or updated list if new fine-mapping is done).
3. **Apply the allele-level predictive model**:

   * For each haplotype, start with baseline Lp(a) predicted from KIV-2 length.
   * Modify baseline by multiplicative effects of causal SNPs/PSVs present on same haplotype.
   * Sum the two haplotype contributions to get predicted Lp(a).
4. **Use predicted Lp(a)** (or residuals if modeling other phenotypes) as the PRS or as part of a multi-trait PRS.

**VNTRwrap LPA-specific Workflow**

**Main Pipeline (generic VNTR handling before LPA branch)**

1. **Count Reads**

   * *Software:* `samtools` 1.17
   * Input: BAM files, VNTR coordinates (from Mukamel TRF)
   * Output: counts of reads overlapping VNTR regions.

2. **Generate Depth Profiles**

   * *Software:* `mosdepth` v0.2.5
   * Input: BAM files + GRCh38 reference
   * Output: per-1kb-window read depth.

3. **Normalize Depth Profiles**

   * *Software:* C++ normalization script (`g++`, Boost 1.58)
   * Input: Repeat mask BED, mosdepth outputs
   * Goal: remove sample-to-sample technical biases.

4. **Find Neighbors**

   * *Software:* C++ neighbor-finding script
   * Input: normalized depth profiles
   * Output: list of most similar samples (for later normalization).

---

**LPA-specific Branch**
Runs **after Step 4** (skips main pipeline Step 5).

**LPA-1. Extract Reference**

* *Software:* `bedtools` 2.30
* Input: hg38/hg37 FASTA + BED of KIV-2 region
* Output: fasta of KIV-2 sequence.

**LPA-2. Realign**

* *Software:* C++ realignment script, `samtools`
* Input: KIV-2 fasta, BAMs
* Output: counts for exon 1A and 1B only.

**LPA-3. Compute Diploid Copy Number**

* *Software:* bash script
* Input: realignment counts + neighbor list
* Method: top 200 neighbors, normalize within-individual & across neighbors.
* Output: per-sample diploid CN for exon1A, exon1B, exon1B\_KIV3, exon1B\_notKIV3.

**LPA-4. (Deprecated)**

* PSV-based subtyping + phasing (Mukamel did this; your PI skipped).

**LPA-5. Estimate Total KIV-2 CN**

* *Software:* Python (pandas)
* Formula: `KIV_CN = 34.9 * exon1A + 5.2 * exon1B - 1`.
